In [1]:
import cv2
import os
import numpy as np

In [2]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
recognizer = cv2.face.LBPHFaceRecognizer_create()

In [3]:
# Function to capture images and store in dataset folder
def capture_images(User):
	# directory to store the captured images
	if not os.path.exists('Faces'):
		os.makedirs('Faces')
  
	cap = cv2.VideoCapture(0)
	count = 0

	while True:
		ret, frame = cap.read()
		gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
		faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

		# Drawing rectangles around the faces
		for (x, y, w, h) in faces:
			cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

			# Storing the captured face images in the Faces folder
			cv2.imwrite(f'Faces/{User}_{count}.jpg', gray[y:y + h, x:x + w])
			count += 1

		cv2.imshow('Capture Faces', frame)

		if cv2.waitKey(1) & 0xFF == ord('q'):
			break

		# Break the loop after capturing a certain number of images
		if count >= 3:
			break

	cap.release()
	cv2.destroyAllWindows()

In [4]:
# Creating the dataset of faces
capture_images('Ann')

In [5]:
# Create the dataset of faces
capture_images('Bell')

In [6]:
label = {'Ann' : 0, 'Bell' : 1}
label

{'Ann': 0, 'Bell': 1}

In [7]:
def train_model(label):
	# Creating lists to store the face samples and their corresponding labels
	faces = []
	labels = []
	
	# Loadig images
	for file_name in os.listdir('Faces'):
		if file_name.endswith('.jpg'):
			name = file_name.split('_')[0]
	
			image = cv2.imread(os.path.join('Faces', file_name))
			gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

			detected_faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

			# Checking if a face is detected
			if len(detected_faces) > 0:
				# Cropping the detected face region
				face_crop = gray[detected_faces[0][1]:detected_faces[0][1] + detected_faces[0][3],
								detected_faces[0][0]:detected_faces[0][0] + detected_faces[0][2]]

				# Appending the face sample and label to the lists
				faces.append(face_crop)
				labels.append(label[name])

	# Training the face recognition model using the faces and labels
	recognizer = cv2.face.LBPHFaceRecognizer_create()
	recognizer.train(faces, np.array(labels))

	# Saving the trained model to a file
	recognizer.save('trained_model.xml')
	return recognizer

# Training the model
Recognizer =train_model(label)
Recognizer

< cv2.face.LBPHFaceRecognizer 0x7f1f06be1af0>

In [8]:
# Function to recognize faces
def recognize_faces(recognizer, label):
	cap = cv2.VideoCapture(0)
	
	# Reverse keys and values in the dictionary
	label_name = {value: key for key, value in label.items()}
	while True:
		ret, frame = cap.read()
		gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
		faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
		
		# Recognizing and labeling the faces
		for (x, y, w, h) in faces:
			# Recognizing the face using the trained model
			label, confidence = recognizer.predict(gray[y:y + h, x:x + w])
			#print(confidence)
			if confidence > 50:
				# Displaying the recognized label and confidence level
				cv2.putText(frame, label_name[label], (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
				cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
			else:
				print('Unrecognized')

		cv2.imshow('Recognize Faces', frame)
  
		if cv2.waitKey(1) & 0xFF == ord('q'):
			break

	cap.release()
	cv2.destroyAllWindows()

In [ ]:
recognize_faces(Recognizer, label)